In [57]:
import pandas
import os
import re

In [58]:
SAMPLE = 'sample_0'
SOLVED_FILE_PATH = 'data/output/solved/'
EQUATIONS_FILE_PATH = ''

In [59]:
ground = pandas.read_json('data/input/draw.json')
ground = ground[['lSolutions', 'iIndex']]
ground = ground.set_index('iIndex')
display(len(ground))
ground.head(1)

999

,lSolutions
iIndex,
397760,[2.14285714286]


In [60]:
solved_majority = pandas.read_json('data/output/solved_majority_eqn/solved_majority.jsonl', lines=True)
solved_majority.head(1)

,solved,flag
0,{a: 1155.00000000000},OK


In [61]:
def extract_actual_answer(answer):
    answer = re.sub(r'\s', "", answer)
    answer = re.sub(r'\s|.*####', "", answer)
    return eval(answer)

def remove_leading_zero(number : str):
    return re.sub(r'^0+', '', number)

def extract_chatgpt_answer(answer_text):
    answer_text = answer_text.lower()
    try:
        answers = re.findall(r'the\s*answer\s*is\s*.*', answer_text)[-1]
        numbers = re.findall(r'\d+\.?\d*', answers)
        fractions = re.findall(r'\d+\/\d+', answers)
        answers = numbers + fractions

        to_return = []
        for answer in answers:
            try: to_return.append(round(eval(remove_leading_zero(answer))))
            except: pass
        return set(to_return)
    except:
        answers = re.findall(r'\d+\.?\d*', answer_text)
        if len(answers) == 0: return set()
        return set([answers[0]])
    
def extract_solved(solved):
    answers = re.findall(r'\d+\.?\d*', solved)
    return set([round(eval(answer), 2) for answer in answers])

def grade_response(row, file_name):
    solved = row[f'{file_name}__solved']
    system = row[f'{file_name}__choices']
    solved = extract_solved(solved)
    system = extract_chatgpt_answer(system)
    chatgpt_answer = extract_chatgpt_answer(row[f'{file_name}__question'])
    actual_answer = set(row['lSolutions'])

    row[f'chatgpt_answer'] = chatgpt_answer
    row[f'solved_answer'] = solved
    row[f'system_answer'] = system

    A = 1 if len(chatgpt_answer.intersection(actual_answer)) > 0 else 0
    C = 1 if len(system.intersection(actual_answer)) > 0 else 0
    D = 1 if len(solved.intersection(actual_answer)) > 0 else 0

    row['A. ChatGPT gets it correct from text to final solution'] = A
    row['B. Sympy gets it correct from equations to final solution'] = 1
    row['C. ChatGPT gets it correct from equations to final solution'] = C
    row['D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT'] = D

    category_array = [A, C, D]

    offset = 8 if len(chatgpt_answer.intersection(solved)) > 0 else 0
    if category_array == [0, 0 , 0]: row['category'] = 1 + offset
    if category_array == [0, 0 , 1]: row['category'] = 2 + offset
    if category_array == [0, 1 , 0]: row['category'] = 3 + offset
    if category_array == [0, 1 , 1]: row['category'] = 4 + offset
    if category_array == [1, 0 , 0]: row['category'] = 5 + offset
    if category_array == [1, 0 , 1]: row['category'] = 6 + offset
    if category_array == [1, 1 , 0]: row['category'] = 7 + offset
    if category_array == [1, 1 , 1]: row['category'] = 8 + offset
    
    return row

In [65]:
for file_path in os.listdir(SOLVED_FILE_PATH):
    solved = pandas.read_json(f"{SOLVED_FILE_PATH}{file_path}", lines=True)
    solved = solved[['question_id', 'question', 'choices', 'solved']]
    solved['choices'] = solved['choices'].apply(lambda row : row[0]['message']['content'])
    file_name = file_path.split('.')[0]
    solved = solved.rename(columns={'question': f'{file_name}__question', 'choices': f'{file_name}__choices', 'solved': f'{file_name}__solved'})
    solved = solved.set_index('question_id')
    ground = ground.join(solved, how='left')
    ground = ground.apply(lambda row : grade_response(row, file_name), axis=1)

# print(len(ground))
ground = ground[~ground.index.duplicated(keep='first')]
ground.head(1)

ValueError: columns overlap but no suffix specified: Index(['sample_0__question', 'sample_0__choices', 'sample_0__solved'], dtype='object')

In [64]:
def label_category(row):
    category = row.name[0]
    return row

values = pandas.DataFrame(ground[[
    'category', 
    'A. ChatGPT gets it correct from text to final solution', 
    'B. Sympy gets it correct from equations to final solution',
    'C. ChatGPT gets it correct from equations to final solution',
    'D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT']].value_counts().sort_index())
values = values.apply(lambda row : label_category(row), axis=1)
values = values[[
    # 'E. Final answer and equations same',
    'count',
    # 'Equations',
    # 'Solving',
    # 'Understandable',
    # 'Theory'
]]
values

,,,,,count
category,A. ChatGPT gets it correct from text to final solution,B. Sympy gets it correct from equations to final solution,C. ChatGPT gets it correct from equations to final solution,D. Gets right answer when ChatGPT transforms text to equations and SymPy solves the equations from chat GPT,
1,0,1,0,0,192
2,0,1,0,1,116
5,1,1,0,0,40
6,1,1,0,1,11
9,0,1,0,0,65
10,0,1,0,1,8
13,1,1,0,0,1
14,1,1,0,1,565


In [ ]:
import xlsxwriter

writer = pandas.ExcelWriter('data/output/xlsx/categories.xlsx', engine='xlsxwriter')
values.to_excel(writer, sheet_name='List')

workbook = writer.book

wrap_format = workbook.add_format()
wrap_format.set_text_wrap()

list_sheet = writer.sheets['List']
list_sheet.set_column('A:H', 15, wrap_format)
pie_chart = workbook.add_chart({'type': 'pie'})

pie_chart.set_style(10)
pie_chart.set_size({'width': 500, 'height': 300})
pie_chart.set_title({'name': f'ChatGPT stats'})
pie_chart.add_series({
    'name': f'ChatGPT stats',
    'categories': f"=Chart!$A$2:$A$10",
    'values': f"=Chart!$G$2:$G$16",
    'points': [
        {'fill': {'color': '#e7366b'}},
        {'fill': {'color': '#29cc7a'}},
        {'fill': {'color': '#ffc619'}},
    ],
    'data_labels': {'value': True, 'category': True, 'position': 'center', 'percentage': True, 'name_font': {'size': 14}}
})
list_sheet.insert_chart('A20', pie_chart, {'x_offset': 0, 'y_offset': 0})
workbook.close()


C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$A$2:$A$10' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "
C:\Users\nocet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xlsxwriter\workbook.py:1660: UserWarning: Unknown worksheet reference 'Chart' in range 'Chart!$G$2:$G$16' passed to add_series()
  warn("Unknown worksheet reference '%s' in range "
